In [11]:
from enum import Enum


class Experiment(Enum):
    TREATMENT = 'treatent'
    CONTROL = 'control'

a = Experiment.TREATMENT
# if a is not None:
#     a.value

In [14]:
a.value

'treatent'

In [12]:
if a:
    print('true')
else:
    print('false')

true


In [ ]:
PHASE = 'train'

In [ ]:
%cd ../run

In [ ]:
# !python -m run.prepare_data dir=local phase=$PHASE

In [ ]:
import subprocess

EXP_NAME = 'exp083'
RUN_NAMES = ['run0'] #, 'run1', 'run2', 'run3', 'run4']
MODEL_NAME = 'best_model.pth' # best_model.pth or latest_model.pth
SCORE_TH = 0.005
DISTANCE = 40
LOW_PASS_FILTER_HOUR = 3

for run_name in RUN_NAMES:
    command = [
        'python',
        '-m',
        'run.inference',
        'dir=local',
        f'+experiment={EXP_NAME}',
        f'weight.run_name={run_name}',
        'batch_size=64',
        f'post_process.score_th={SCORE_TH}',
        f'post_process.distance={DISTANCE}',
        f'post_process.low_pass_filter_hour={LOW_PASS_FILTER_HOUR}',
        f'phase={PHASE}',
    ]
    subprocess.run(command)

In [ ]:
EXP_NAME = 'exp085'

for run_name in RUN_NAMES:
    command = [
        'python',
        '-m',
        'run.inference',
        'dir=local',
        f'+experiment={EXP_NAME}',
        f'weight.run_name={run_name}',
        'batch_size=64',
        f'post_process.score_th={SCORE_TH}',
        f'post_process.distance={DISTANCE}',
        f'post_process.low_pass_filter_hour={LOW_PASS_FILTER_HOUR}',
        f'phase={PHASE}',
    ]
    subprocess.run(command)

In [ ]:
import numpy as np

preds_list = []
EXP_NAME = 'exp083'
for run_name in RUN_NAMES:
    preds = np.load(f"../output/inference/{EXP_NAME}/{run_name}/preds.npy")
    preds_list.append(preds)
keys1 = np.load(f"../output/inference/{EXP_NAME}/{run_name}/keys.npy")

preds1 = np.zeros(preds_list[0].shape)
for i in range(len(RUN_NAMES)):
    preds1 += preds_list[i]
# 単純な平均を取る
preds1 /= len(RUN_NAMES)

In [ ]:
preds_list = []
EXP_NAME = 'exp085'
for run_name in RUN_NAMES:
    preds = np.load(f"../output/inference/{EXP_NAME}/{run_name}/preds.npy")
    preds_list.append(preds)
keys2 = np.load(f"../output/inference/{EXP_NAME}/{run_name}/keys.npy")

preds2 = np.zeros(preds_list[0].shape)
for i in range(len(RUN_NAMES)):
    preds2 += preds_list[i]
# 単純な平均を取る
preds2 /= len(RUN_NAMES)

In [ ]:
import pandas as pd

gt_df = pd.read_csv("../data/child-mind-institute-detect-sleep-states/train_events.csv")
gt_df = gt_df.dropna().reset_index(drop=True)

In [ ]:
from run.inference import make_submission
from src.utils.metrics import event_detection_ap

sub_df = make_submission(keys1, preds1, SCORE_TH,DISTANCE, LOW_PASS_FILTER_HOUR)
score = event_detection_ap(gt_df, sub_df.to_pandas())
score

In [ ]:
sub_df = make_submission(keys2, preds2, SCORE_TH,DISTANCE, LOW_PASS_FILTER_HOUR)
score = event_detection_ap(gt_df, sub_df.to_pandas())
score

In [ ]:
n_classes = preds2.shape[2]
chunk_size = preds2.shape[1]

series_ids1 = np.array(list(map(lambda x: x.split("_")[0], keys1)))
series_ids2 = np.array(list(map(lambda x: x.split("_")[0], keys2)))
unique_series_ids = np.unique(series_ids1)
preds_list = []
max_length = 0
for series_id in unique_series_ids:
    series_idx1 = np.where(series_ids1 == series_id)[0]
    series_idx2 = np.where(series_ids2 == series_id)[0]
    series_preds1 = preds1[series_idx1].reshape(-1, n_classes)
    series_preds2 = preds2[series_idx2].reshape(-1, n_classes)
    min_duration = min(len(series_preds1), len(series_preds2))
    series_preds1 = series_preds1[:min_duration]
    series_preds2 = series_preds2[:min_duration]

    series_preds = series_preds1 * 0.5 + series_preds2 * 0.5
    preds_list.append(series_preds)
    max_length = max(max_length, len(series_preds))

# Pad each element to the maximum length
padded_preds_list = [np.pad(x, ((0, max_length - x.shape[0]), (0, 0)), 'constant', constant_values=0) for x in preds_list]
preds = np.stack(padded_preds_list)

In [ ]:
preds.shape

In [ ]:
keys = unique_series_ids

In [ ]:
from run.inference import make_submission

sub_df = make_submission(keys, preds, SCORE_TH,DISTANCE, LOW_PASS_FILTER_HOUR)
sub_df.head()

In [ ]:
from src.utils.metrics import event_detection_ap

score = event_detection_ap(gt_df, sub_df.to_pandas())
score